REQUIREMENTS

In [9]:
pip install xgboost
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importing Libraries

import json
import string
import random 

import nltk
nltk.download('omw-1.4')
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
nltk.download("punkt")
nltk.download("wordnet")

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm 
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn import svm
from sklearn.svm import SVC

# Loading The Dataset 
df = pd.read_csv('Dataset_Cognitive_Distortions.tsv', sep='\t', header=0)
#df = df.drop(df.columns[[2, 3, 4]], axis=1)  # df.columns is zero-based pd.Index
df.shape

#Data Preprocessing 
df['Phrase'] = df['Phrase'].str.lower()


# Divide data between training and test data
X = df['Phrase']
y = df['Cognitive Distortion']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .8)


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#Best model 
topiclassification = Pipeline([
   ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=0)),
   ('classifier', MultinomialNB())
])

search_space = [{'classifier': [MultinomialNB()],
                               'classifier__alpha': [0.01, 0.1, 0.3, 0.5, 1.0, 10, ],
                               'classifier__fit_prior': [True, False]
                 },
                {'smote__k_neighbors': [2,3,4,5,6,7,8,9,10, ]}, 
                
                {'classifier': [LogisticRegression(multi_class='multinomial', random_state=0)],
                               'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none']                
                 },
                {'classifier': [svm.SVC(probability = True)],
                               'classifier__decision_function_shape': ['ovo', 'ovr']
                },
                {'classifier':  [XGBClassifier(objective='multi:softmax')]
                },
                {'classifier':  [AdaBoostClassifier(n_estimators=5000)]
                },
                {'classifier':  [RandomForestClassifier(random_state=0)]
                },
                {'classifier':  [LGBMClassifier(random_state=0)]
                },
                { 'classifier': [KNeighborsClassifier()],
                                'classifier__n_neighbors': [5,6,7,8,9,10, ],
                                'classifier__weights': ['uniform', 'distance']
                },
                ]

best_model = GridSearchCV(estimator=topiclassification, param_grid=search_space, n_jobs=50, cv=10, verbose=5)
best_model.fit(X_train, y_train)

print('Train Accuracy : %.3f'%best_model.best_estimator_.score(X_train, y_train))
print('Test Accuracy : %.3f'%best_model.best_estimator_.score(X_test, y_test))
print('Best Accuracy Through Grid Search : %.3f'%best_model.best_score_)
print('Best Parameters : ',best_model.best_params_)

Fitting 10 folds for each of 43 candidates, totalling 430 fits


c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packages\imblearn\pipeline.py", line 297, in fit
    self._final_estimator.fit(Xt, yt, **fit_params_last_step)
  File "c:\Users\Usuario\Desktop\Kai-SelfCare-Chatbot\.venv\lib\site-packag

Train Accuracy : 0.998
Test Accuracy : 0.622
Best Accuracy Through Grid Search : 0.675
Best Parameters :  {'classifier': LogisticRegression(multi_class='multinomial', penalty='none', random_state=0), 'classifier__penalty': 'none'}
